<a href="https://colab.research.google.com/github/Huntfalow123/data2020/blob/master/DataTransform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
from google.colab import drive
import pandas as pd
import io
!pip install country_converter
!pip install pycountry
import pycountry
import country_converter as coco
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
Global_Mobility = pd.read_csv('/content/drive/My Drive/Datasets/00 Core/Community mobility reports/Global_Mobility_Report.csv')
Covid_Response = pd.read_csv('/content/drive/My Drive/Datasets/00 Core/Novel Coronavirus 2019 datasets/covid_19_data.csv')
Government_Reponse = pd.read_csv('/content/drive/My Drive/Datasets/00 Core/Oxford Covid-19 Government Response Tracker/OxCGRT_latest.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
Global_Mobility.dropna(subset=['country_region_code', 'date'],inplace=True)
countries = {}
for country in pycountry.countries:
    countries[country.alpha_2] = country.alpha_3
Global_Mobility['country_region_code'].replace(countries, inplace=True)
Global_Mobility = Global_Mobility[['country_region_code','date','retail_and_recreation_percent_change_from_baseline','grocery_and_pharmacy_percent_change_from_baseline','parks_percent_change_from_baseline','transit_stations_percent_change_from_baseline','workplaces_percent_change_from_baseline','residential_percent_change_from_baseline']]
print(Global_Mobility['country_region_code'].drop_duplicates().isna().sum())

0


In [55]:
Global_Mobility.update(Global_Mobility.sort_values(['country_region_code','date'],ascending=True).groupby('country_region_code').ffill())

In [48]:
print(Global_Mobility.isna().sum())

country_region_code                                     0
date                                                    0
retail_and_recreation_percent_change_from_baseline      0
grocery_and_pharmacy_percent_change_from_baseline       3
parks_percent_change_from_baseline                    220
transit_stations_percent_change_from_baseline         228
workplaces_percent_change_from_baseline                 2
residential_percent_change_from_baseline              954
dtype: int64


In [71]:
Government_Reponse

,CountryName,CountryCode,RegionName,RegionCode,Date,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,C3_Cancel public events,C3_Flag,C4_Restrictions on gatherings,C4_Flag,C5_Close public transport,C5_Flag,C6_Stay at home requirements,C6_Flag,C7_Restrictions on internal movement,C7_Flag,C8_International travel controls,E1_Income support,E1_Flag,E2_Debt/contract relief,E3_Fiscal measures,E4_International support,H1_Public information campaigns,H1_Flag,H2_Testing policy,H3_Contact tracing,H4_Emergency investment in healthcare,H5_Investment in vaccines,M1_Wildcard,ConfirmedCases,ConfirmedDeaths,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
0,Aruba,ABW,NaN,NaN,20200101,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
1,Aruba,ABW,NaN,NaN,20200102,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
2,Aruba,ABW,NaN,NaN,20200103,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
3,Aruba,ABW,NaN,NaN,20200104,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
4,Aruba,ABW,NaN,NaN,20200105,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66511,Zimbabwe,ZWE,NaN,NaN,20200928,2.0,1.0,1.0,1.0,2.0,1.0,3.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,4.0,1.0,1.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,0.0,0.0,NaN,7812.0,227.0,76.85,76.85,84.52,84.52,63.46,63.46,70.45,70.45,25.0,25.0
66512,Zimbabwe,ZWE,NaN,NaN,20200929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7816.0,228.0,NaN,76.85,NaN,84.52,NaN,63.46,NaN,70.45,NaN,25.0
66513,Zimbabwe,ZWE,NaN,NaN,20200930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7837.0,228.0,NaN,76.85,NaN,84.52,NaN,63.46,NaN,70.45,NaN,25.0
66514,Zimbabwe,ZWE,NaN,NaN,20201001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7838.0,228.0,NaN,76.85,NaN,84.52,NaN,63.46,NaN,70.45,NaN,25.0


In [75]:
Table = ['C1_Flag','C2_Flag','C3_Flag','C4_Flag','C5_Flag','C6_Flag','C7_Flag','C8_International travel controls']
Gov_Res = Government_Reponse[['CountryCode']].drop_duplicates()
for col in Table:
  SchoolC = Government_Reponse[['CountryCode','Date',col]]
  SchoolC.dropna(inplace=True)
  SchoolC['Date'] = pd.to_datetime(SchoolC['Date'],format="%Y%m%d")
  SchoolC = SchoolC.sort_values(['CountryCode','Date'],ascending=True).groupby('CountryCode').first().reset_index()
  SchoolC[col+'_Date'] = SchoolC['Date'] 
  SchoolC = SchoolC[['CountryCode',col+'_Date']]
  Gov_Res = pd.merge(Gov_Res,SchoolC,how = 'left',on='CountryCode')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipyk

In [77]:
Covid_Response

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
116800,116801,09/23/2020,Zaporizhia Oblast,Ukraine,2020-09-24 04:23:38,3149.0,49.0,1158.0
116801,116802,09/23/2020,Zeeland,Netherlands,2020-09-24 04:23:38,1270.0,72.0,0.0
116802,116803,09/23/2020,Zhejiang,Mainland China,2020-09-24 04:23:38,1282.0,1.0,1272.0
116803,116804,09/23/2020,Zhytomyr Oblast,Ukraine,2020-09-24 04:23:38,5191.0,92.0,2853.0


In [90]:
Covid_Response.dropna(subset=['Country/Region', 'ObservationDate'],inplace=True)
countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_3
    Covid_Response['Country/Region'].replace({'Mainland China':'China','Taiwan':'Taiwan, Province of China','Russia':'Russian Federation','Macau':'Macao',
                                              'South Korea':'Korea, Republic of','Vietnam':'Viet Nam','Syria':'Syrian Arab Republic'}, inplace=True)
Covid_Response['Country/Region'].replace(countries, inplace=True)
Covid_Response = Covid_Response[['Country/Region','ObservationDate','Confirmed']]
Covid_Response = Covid_Response.groupby(['Country/Region','ObservationDate']).sum().reset_index()
Covid_Response[Covid_Response['Country/Region'] == 'AUS']

,Country/Region,ObservationDate,Confirmed
1661,AUS,01/23/2020,0.0
1662,AUS,01/25/2020,4.0
1663,AUS,01/26/2020,4.0
1664,AUS,01/27/2020,5.0
1665,AUS,01/28/2020,5.0
...,...,...,...
1900,AUS,09/19/2020,26898.0
1901,AUS,09/20/2020,26912.0
1902,AUS,09/21/2020,26942.0
1903,AUS,09/22/2020,26972.0
